In [1]:
# https://gee-community-catalog.org/projects/annual_nlcd/#earth-engine-snippet
import ee
import xarray as xr
import pandas as pd
import numpy as np
import dask.array as da
import xee
from bbox import * 
from datetime import datetime
# Trigger the authentication flow
ee.Authenticate()

# Initialize the Earth Engine library
ee.Initialize(project='satellite-modeling')

# Define the rectangle that encompasses Louisiana and Texas
bbox = ee.Geometry.Rectangle([lon_min, lat_min, lon_max, lat_max])

start_date = '2024-12-01T00:00'
# start_date = '2024-09-20T00:00'

end_date = datetime.now().strftime('%Y-%m-%dT%H:%M')

In [2]:
no2_hours = pd.read_csv("../../data/tempo_data/no2_file_hours.csv")
no2_hours=list(no2_hours['time_hr_ct'])
time_list = np.array(no2_hours, dtype="datetime64[ns]")

In [ ]:
##################################################
# Load RTMA Data
##################################################

# List of climate variables to download from NOAA RTMA
variables = ['PRES', 'TMP', 'UGRD', 'VGRD','DPT','SPFH','WDIR', 'WIND', 'GUST', 'VIS', 'TCDC', 'ACPC01']
# Load NOAA hourly data and filter by region and date
dataset = ee.ImageCollection("NOAA/NWS/RTMA") \
            .filterDate(start_date, end_date) \
            .filterBounds(bbox)  \
            .select(variables)

ds = xr.open_dataset(dataset, engine='ee', crs='EPSG:4326', scale=0.01)
# Filter the dataset based on the latitude and longitude bounds
ds = ds.sel(lat=slice(lat_min, lat_max), lon=slice(lon_min, lon_max))

##################################################
# Change Timestamps
##################################################
# Recreate UTC Time Stamps
ds_time_list = ds["time"].values.tolist()
timestamps_utc = pd.to_datetime(ds_time_list)
timestamps_utc = timestamps_utc.tz_localize('UTC')

# # Add 1 hour to each timestamp so that it contains weather data from prior hour
# timestamps_adjusted= = timestamps_utc + pd.Timedelta(hours=1)
timestamps_adjusted=timestamps_utc

# Convert to Central Time (handles daylight saving time automatically)
timestamps_central = timestamps_adjusted.tz_convert('America/Chicago')
timestamps_central = timestamps_central.tz_localize(None)
timestamps_central=np.array(timestamps_central, dtype="datetime64[ns]")
# Replace the time dimension
ds = ds.assign_coords(time=timestamps_central)

# Remove duplicate hour from time change
ds = ds.isel(time=ds.get_index("time").duplicated() == False)

# Find dates included in both sat and weather data
seta = set(timestamps_central)
setb = set(time_list)
union_set = seta & setb
full_time_list = list(union_set)
ds = ds.sel(time=full_time_list)

##################################################
# Save monthly files
##################################################
# Get unique months
unique_months = sorted(set(date.astype('datetime64[M]') for date in full_time_list))

for i in range(3,len(unique_months)):
    print(i)
    month=unique_months[i]
    month_str=str(month)
    dates_in_month = [date for date in full_time_list if date.astype('datetime64[M]') == month]
    ds_month = ds.sel(time=dates_in_month)
    
    ds_month = ds_month.chunk({'time': 1}).persist()
    print('**')

    ds_month.to_netcdf(f'../../data/weather_data_samehr/weather_data_{month_str}.nc', engine='h5netcdf')

3
**
4
**
5
**
6
**


In [ ]:
##################################################
# Load Radiation Data
##################################################

# List of climate variables to download from NOAA RTMA
variables = ['LWGNT', 'SWGNT']
# Load NOAA hourly data and filter by region and date
dataset = ee.ImageCollection("NASA/GSFC/MERRA/rad/2") \
            .filterDate(start_date, end_date) \
            .filterBounds(bbox)  \
            .select(variables)

ds = xr.open_dataset(dataset, engine='ee', crs='EPSG:4326', scale=0.01)
# Filter the dataset based on the latitude and longitude bounds
ds = ds.sel(lat=slice(lat_min, lat_max), lon=slice(lon_min, lon_max))

##################################################
# Change Timestamps
##################################################
# Recreate UTC Time Stamps
ds_time_list = ds["time"].values.tolist()
timestamps_utc = pd.to_datetime(ds_time_list)
timestamps_utc = timestamps_utc.tz_localize('UTC')

# # Add 1 hour to each timestamp so that it contains weather data from prior hour
# timestamps_adjusted= = timestamps_utc + pd.Timedelta(hours=1)
timestamps_adjusted=timestamps_utc

# Convert to Central Time (handles daylight saving time automatically)
timestamps_central = timestamps_adjusted.tz_convert('America/Chicago')
timestamps_central = timestamps_central.tz_localize(None)
timestamps_central=np.array(timestamps_central, dtype="datetime64[ns]")
# Replace the time dimension
ds = ds.assign_coords(time=timestamps_central)

# Remove duplicate hour from time change
ds = ds.isel(time=ds.get_index("time").duplicated() == False)

# Find dates included in both sat and weather data
seta = set(timestamps_central)
setb = set(time_list)
union_set = seta & setb
full_time_list = list(union_set)
ds = ds.sel(time=full_time_list)
unique_months = sorted(set(date.astype('datetime64[M]') for date in full_time_list))


for i in range(len(unique_months)):
    print(i)
    month=unique_months[i]
    month_str=str(month)
    dates_in_month = [date for date in full_time_list if date.astype('datetime64[M]') == month]
    ds_month = ds.sel(time=dates_in_month)
    
    ds_month = ds_month.chunk({'time': 1}).persist()
    print('**')

    ds_month.to_netcdf(f'../../data/weather_data_samehr/radiation_data_{month_str}.nc', engine='h5netcdf')

0
**
1
**
2
**
3
**
4
**
5
**
6
**
